Install Ngrok, SQLAlchemy, Flask-Session, & setup token.

Mount Google Drive for File Access.

In [ ]:
!pip install pyngrok
!pip install flask_sqlalchemy
!pip install Flask-Session

import os

# Prompt user for the Ngrok auth token
ngrok_token = input("Enter your Ngrok auth token: ")

# Set it as an environment variable
os.environ["NGROK_AUTH_TOKEN"] = ngrok_token

# Verify the token is stored
print("Ngrok auth token stored successfully!")

# Access it without exposing it in the code
auth_token = os.getenv("NGROK_AUTH_TOKEN")

from google.colab import drive
drive.mount('/content/drive')

Latest Server Code

In [ ]:
from flask import Flask, request, jsonify, render_template, session
from pyngrok import ngrok
from werkzeug.security import generate_password_hash, check_password_hash
from flask_session import Session
import sqlite3
import os
from google.colab import drive
import random

# Define Paths
base_path = "/content/drive/My Drive/colab-web-app/"
template_path = f"{base_path}/templates"
static_path = f"{base_path}/static"
db_path = "/content/drive/MyDrive/colab-web-app/database/user_database.db"

# Flask App Setup
app = Flask(__name__, template_folder=template_path, static_folder=static_path)

# Flask-Session Configuration
app.config["SESSION_TYPE"] = "filesystem"
app.config["SECRET_KEY"] = "supersecretkey"  # Replace with a secure key
Session(app)

# Mount Google Drive (for database persistence)
drive.mount('/content/drive')

# Database Connection Function
def get_db_connection():
    conn = sqlite3.connect(db_path)
    conn.row_factory = sqlite3.Row
    return conn

# Home Page Route (Shows Logged-In User)
@app.route('/')
def home():
    username = session.get("username", "Guest")
    return render_template('index.html', username=username)

@app.route("/", methods=["POST"])
def process_input():
    user_input = request.form.get("user_input")
    return jsonify({"message": f"You entered: {user_input}"})

@app.route('/register_page')
def show_register_form():
    return render_template('register.html')

# User Registration API
@app.route('/register_user', methods=['POST'])
def register_user():
    data = request.json
    username, email, password = data["username"], data["email"], data["password"]

    # Hash password before storing
    hashed_password = generate_password_hash(password)

    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute("INSERT INTO users (username, email, password) VALUES (?, ?, ?)",
                       (username, email, hashed_password))
        conn.commit()
        conn.close()
        return jsonify({"message": "User registered successfully!"})
    except sqlite3.IntegrityError:
        return jsonify({"error": "Username or Email already exists!"}), 400


# Retrieve Users Endpoint
@app.route('/get_users', methods=['GET'])
def get_users():
    try:
        # Connect to the database using the defined path
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Fetch user data (id, username, email)
        cursor.execute("SELECT id, username, email FROM users")
        users = cursor.fetchall()
        conn.close()

        # Build a list that includes id, username, and email for each user
        users_list = [{"id": user[0], "username": user[1], "email": user[2]} for user in users]
        return jsonify(users_list)
    except Exception as e:
        # Return any errors that might occur during connection or fetching data
        return jsonify({"error": str(e)}), 500


@app.route('/login_page')
def show_login_form():
    return render_template('login.html')

# User Login Endpoint (Pre-Session)
# @app.route('/login', methods=['POST'])
# def login():
#     data = request.json
#     email, password = data["email"], data["password"]

#     conn = get_db_connection()
#     cursor = conn.cursor()
#     cursor.execute("SELECT * FROM users WHERE email = ?", (email,))
#     user = cursor.fetchone()
#     conn.close()

#     if user and check_password_hash(user["password"], password):
#         session["user_id"] = user["id"]
#         session["username"] = user["username"]
#         return jsonify({"message": "Login successful!", "username": user["username"]})

#     return jsonify({"error": "Invalid credentials!"}), 401

@app.route('/login', methods=['POST'])
def login():
    """
    Expects JSON: { email, password }
    On success:
      • stores user_id & username in session
      • returns { message, username }
    On failure:
      • 401 + { error }
    """
    data = request.get_json(force=True)
    email = data.get("email", "").strip()
    password = data.get("password", "")

    # fetch user row
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute("SELECT id, username, password FROM users WHERE email = ?", (email,))
    user = cur.fetchone()
    conn.close()

    # verify & set session
    if user and check_password_hash(user["password"], password):
        session["user_id"]  = user["id"]
        session["username"] = user["username"]
        return jsonify({
            "message":  "Login successful!",
            "username": user["username"]
        })

    # bad creds
    return jsonify({"error": "Invalid credentials!"}), 401

# TEST LOGOUT
# Logout Endpoint
# @app.route('/logout', methods=['GET'])
# def logout():
#     session.pop("user_id", None)
#     session.pop("username", None)
#     return jsonify({"message": "Logged out successfully!"})

GOODBYES = [
    "Vibe offline, but spirit stays online. See you next time!",
    "Your coding couch misses you already. Drop by again soon!",
    "Our pixels part today, but the code remains. Until we ping again!",
]

# Pre-Session Code
# @app.route('/logout')
# def logout():
#     # Pick one random goodbye message
#     message = random.choice(GOODBYES)
#     return render_template('logout.html', goodbye=message)

@app.route('/logout', methods=['GET', 'POST'])
def logout():
    """
    Clears the session.
    If this was an AJAX / POST logout, returns JSON.
    Otherwise renders the friendly logout.html page.
    """
    # 1) clear everything
    session.pop("user_id", None)
    session.pop("username", None)

    # 2) detect JSON/JS call
    if request.is_json or request.method == 'POST':
        return jsonify({"message": "Logged out successfully."})

    # 3) direct GET → show goodbye page
    goodbye = random.choice(GOODBYES)
    return render_template('logout.html', goodbye=goodbye)

# Ngrok Authentication & Tunnel Setup
auth_token = os.getenv("NGROK_AUTH_TOKEN")
if auth_token:
    ngrok.set_auth_token(auth_token)

public_url = ngrok.connect(8000)
print(f'Access the web app here: {public_url}')

# Run Flask Server
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8000)
